In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('allegations_20200726939.csv')

In [3]:
len(df)

33358

In [4]:
df.sample()

,unique_mos_id,first_name,last_name,command_now,complaint_id,month_received,year_received,month_closed,year_closed,command_at_incident,...,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
15099,23234,Michael,Fratangelo,INT OAS,38296,10,2017,7,2018,WARRSEC,...,30,NaN,NaN,NaN,Abuse of Authority,Search of Premises,44.0,Execution of arrest/bench warrant,No arrest made or summons issued,Substantiated (Formalized Training)


In [21]:
df.unique_mos_id.value_counts().reset_index()

,index,unique_mos_id
0,25861,75
1,18731,75
2,19489,73
3,18530,73
4,18589,72
...,...,...
3991,11517,1
3992,26350,1
3993,22637,1
3994,22280,1
